Class: DSC 670<br>
Name: Eros Perez<br>
Week: 12<br>
Assignment: WFM x AI App

### Code For Streamlit LLM APP

This code was copy and pasted from a python file I created in Visual Studio Code for the purpose of transforming into PDF for ease of visibility. 

In [ ]:
import os
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
from openai import OpenAI
import re

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

st.set_page_config(page_title="WFM Forecast Visualizer", layout="centered")
st.title("📈 WFM Forecast Visualizer")
st.write("Upload weekly volume CSV. The AI will analyze trends and forecast 2025.")

# Upload CSV
uploaded_file = st.file_uploader("Upload your CSV file", type=["csv"])

if uploaded_file:
    try:
        df = pd.read_csv(uploaded_file, parse_dates=["week_start"])
        st.success("CSV uploaded successfully!")

        st.subheader("📊 Uploaded Data Preview")
        st.dataframe(df.head())

        # Format for prompt
        df_text = "week_start,volume\n" + "\n".join([
            f"{row['week_start'].date()},{int(row['volume'])}" for _, row in df.iterrows()
        ])

        prompt = (
            "You are a WFM forecasting expert.\n\n"
            "Here is weekly contact center volume data in CSV format:\n\n"
            f"{df_text}\n\n"
            "Please forecast weekly volume for 2025. Generate realistic actuals. Output your result in this format:\n"
            "Week Date | Forecast | Actuals\n\n"
            "Then calculate:\nMAPE:\nMAE:\nWeekly Variance Percentage:\nTotal Average Variance Percentage:\n\n"
            "Finish with a paragraph summarizing the trend, seasonality, cycles, and call out outliers you observe using percentages, "
            "then add how this impacts staffing levels.\n\n"
        )

        with st.spinner("Analyzing with LLM..."):
            response = client.chat.completions.create(
                model="ft:gpt-3.5-turbo-1106:personal:wfm-predictor:BVlu4dkL",
                messages=[
                    {"role": "system", "content": "You are a WFM forecasting expert."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=4096
            )
            result = response.choices[0].message.content

        # Extract forecast table from response
        forecast_lines = []
        summary_lines = []
        table_started = False

        for line in result.splitlines():
            if "|" in line and "Week Date" not in line:
                table_started = True
                forecast_lines.append(line.strip())
            elif table_started and (":" in line or line.strip() == ""):
                break

        # Parse the forecast lines
        forecast_data = []
        for line in forecast_lines:
            try:
                parts = [p.strip() for p in line.split("|")]
                forecast_data.append({
                    "Week Date": pd.to_datetime(parts[0]),
                    "Forecast": int(parts[1]),
                    "Actual": int(parts[2])
                })
            except:
                continue

        forecast_df = pd.DataFrame(forecast_data)

        # Extract summary and metrics
        summary_start = result.find("MAPE:")
        summary_text = result[summary_start:].strip()

        # Plot forecast vs actuals
        st.subheader("📉 Forecast vs Actuals Chart")
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.plot(forecast_df["Week Date"], forecast_df["Forecast"], label="Forecast", marker='o')
        ax.plot(forecast_df["Week Date"], forecast_df["Actual"], label="Actual", marker='x')
        ax.set_title("Forecast vs Actual Weekly Volume")
        ax.set_xlabel("Week Date")
        ax.set_ylabel("Volume")
        ax.legend()
        plt.xticks(rotation=45)
        st.pyplot(fig)

        # Display full response text (optional)
        st.subheader("📋 Forecasting Metrics and Analysis")
        st.text_area("Full Summary from LLM", summary_text, height=300)

    except Exception as e:
        st.error(f"Something went wrong: {e}")
